[Starter code used](https://www.kaggle.com/rsmits/keras-efficientnet-b3-training-inference)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import tensorflow as tf
import keras
from keras.models import Model, Input
from keras.layers import Input, Dense, Lambda
from keras.optimizers import Adam
from keras.callbacks import Callback, ModelCheckpoint

import cv2 
from math import ceil
import gc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Using TensorFlow backend.


/kaggle/input/weights-v1/Train1_model_13.h5
/kaggle/input/weights-v1/Train1_model_12.h5
/kaggle/input/weights-v1/Train1_model_14.h5
/kaggle/input/weights-v1/Train1_model_11.h5
/kaggle/input/bengaliai-cv19/test_image_data_0.parquet
/kaggle/input/bengaliai-cv19/sample_submission.csv
/kaggle/input/bengaliai-cv19/test_image_data_3.parquet
/kaggle/input/bengaliai-cv19/train_image_data_0.parquet
/kaggle/input/bengaliai-cv19/test_image_data_1.parquet
/kaggle/input/bengaliai-cv19/train_image_data_2.parquet
/kaggle/input/bengaliai-cv19/train_image_data_3.parquet
/kaggle/input/bengaliai-cv19/test_image_data_2.parquet
/kaggle/input/bengaliai-cv19/train_image_data_1.parquet
/kaggle/input/bengaliai-cv19/class_map.csv
/kaggle/input/bengaliai-cv19/test.csv
/kaggle/input/bengaliai-cv19/train.csv


In [2]:
# Constants
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

HEIGHT = 137
WIDTH = 236
HEIGHT_NEW = 128
WIDTH_NEW = 128
CHANNELS = 3

BATCH_SIZE = 32 
TRAIN_DIR = '.'
TEST_SIZE = 1./8
EPOCHS = 40

RUN_NAME = 'Train1_'
PLOT_NAME1 = 'Train1_LossAndAccuracy.png'
PLOT_NAME2 = 'Train1_Recall.png'

# **Image Preprocessing**

[ROI resize](https://www.kaggle.com/shawon10/bangla-graphemes-image-processing-deep-cnn)

In [3]:
def resize_roi(image, size=128):
    resized = {}
    resize_size=size
    
    _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

    idx = 0 
    ls_xmin = []
    ls_ymin = []
    ls_xmax = []
    ls_ymax = []
    for cnt in contours:
        idx += 1
        x,y,w,h = cv2.boundingRect(cnt)
        ls_xmin.append(x)
        ls_ymin.append(y)
        ls_xmax.append(x + w)
        ls_ymax.append(y + h)
    xmin = min(ls_xmin)
    ymin = min(ls_ymin)
    xmax = max(ls_xmax)
    ymax = max(ls_ymax)

    roi = image[ymin:ymax,xmin:xmax]
    resized_roi = cv2.resize(roi, (resize_size, resize_size), interpolation=cv2.INTER_AREA)

    return resized_roi

# Image Prep
def resize_image(img, WIDTH_NEW, HEIGHT_NEW):
    # Invert
    img = 255 - img

    # Normalize
    img = (img * (255.0 / img.max())).astype(np.uint8)

    # Reshape
    img = img.reshape(HEIGHT, WIDTH)
    img = resize_roi(img)

    return img

# **Model Definition**

In [4]:
# Generalized mean pool - GeM
gm_exp = tf.Variable(3.0, dtype = tf.float32)
def generalized_mean_pool_2d(X):
    pool = (tf.reduce_mean(tf.abs(X**(gm_exp)), 
                        axis = [1, 2], 
                        keepdims = False) + 1.e-7)**(1./gm_exp)
    return pool

def create_model(input_shape):
    # Input Layer
    input_tensor = Input(shape = input_shape)
    
    # Create and Compile Model and show Summary
    x_model = keras.applications.nasnet.NASNetMobile(input_tensor=input_tensor, include_top=False, weights=None, pooling=None, classes=None)
    
    # UnFreeze all layers
    for layer in x_model.layers:
        layer.trainable = True
    
    # GeM
    lambda_layer = Lambda(generalized_mean_pool_2d)
    lambda_layer.trainable_weights.extend([gm_exp])
    x = lambda_layer(x_model.output)
    
    # multi output
    grapheme_root = Dense(168, activation = 'softmax', name = 'root')(x)
    vowel_diacritic = Dense(11, activation = 'softmax', name = 'vowel')(x)
    consonant_diacritic = Dense(7, activation = 'softmax', name = 'consonant')(x)

    # model
    model = Model(inputs = x_model.input, outputs = [grapheme_root, vowel_diacritic, consonant_diacritic])

    return model

In [5]:
model = create_model((HEIGHT_NEW, WIDTH_NEW, CHANNELS))
# Compile Model
model.load_weights('/kaggle/input/weights-v1/Train1_model_14.h5')
# Model Summary
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 63, 63, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 63, 63, 32)   128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 63, 63, 32)   0           stem_bn1[0][0]                   
____________________________________________________________________________________________

# **Testing**

In [6]:
class TestDataGenerator(keras.utils.Sequence):
    def __init__(self, X, batch_size = 16, img_size = (512, 512, 3), *args, **kwargs):
        self.X = X
        self.indices = np.arange(len(self.X))
        self.batch_size = batch_size
        self.img_size = img_size
                    
    def __len__(self):
        return int(ceil(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(indices)
        return X
    
    def __data_generation(self, indices):
        X = np.empty((self.batch_size, *self.img_size))
        
        for i, index in enumerate(indices):
            image = self.X[index]
            image = np.stack((image,)*CHANNELS, axis=-1)
            image = image.reshape(-1, HEIGHT_NEW, WIDTH_NEW, CHANNELS)
            
            X[i,] = image
        
        return X

In [7]:
# Parquet file directory
parquetdir = '/kaggle/input/bengaliai-cv19'

# Create Submission File
tgt_cols = ['grapheme_root','vowel_diacritic','consonant_diacritic']

# Create Predictions
row_ids, targets = [], []

# Loop through Test Feather files (X)
for i in range(0, 4):
    # Test Files Placeholder
    test_files = []

    # Read Feather file
    df = pd.read_parquet(parquetdir+'/test_image_data_'+str(i)+'.parquet')
    
    # Get Image Id values
    image_ids = df['image_id'].values 
    # Drop Image_id column
    df = df.drop(['image_id'], axis = 1)

    # Loop over rows in Dataframe and generate images 
    X = []
    for image_id, index in zip(image_ids, range(df.shape[0])):
        test_files.append(image_id)
        X.append(resize_image(df.loc[df.index[index]].values, WIDTH_NEW, HEIGHT_NEW))

    # Data_Generator
    data_generator_test = TestDataGenerator(X, batch_size = BATCH_SIZE, img_size = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
    
    # Predict with model
    pred = model.predict_generator(data_generator_test, verbose = 1)

    # Loop over Preds    
    for i, image_id in zip(range(len(test_files)), test_files):        
        for subi, col in zip(range(len(pred)), tgt_cols):
            # Set Prediction
            row_ids.append(str(image_id)+'_'+col)
            targets.append(pred[subi])
    
    # Cleanup
    del df
    gc.collect()

1/1 [==============================] - 1s 718ms/step


# **Submission**

In [8]:
submit_df = pd.DataFrame({'row_id':row_ids,'target':targets}, columns = ['row_id','target'])
submit_df.to_csv('submission.csv', index = False)
print(submit_df.head(40))

                         row_id  \
0          Test_0_grapheme_root   
1        Test_0_vowel_diacritic   
2    Test_0_consonant_diacritic   
3          Test_1_grapheme_root   
4        Test_1_vowel_diacritic   
5    Test_1_consonant_diacritic   
6          Test_2_grapheme_root   
7        Test_2_vowel_diacritic   
8    Test_2_consonant_diacritic   
9          Test_3_grapheme_root   
10       Test_3_vowel_diacritic   
11   Test_3_consonant_diacritic   
12         Test_4_grapheme_root   
13       Test_4_vowel_diacritic   
14   Test_4_consonant_diacritic   
15         Test_5_grapheme_root   
16       Test_5_vowel_diacritic   
17   Test_5_consonant_diacritic   
18         Test_6_grapheme_root   
19       Test_6_vowel_diacritic   
20   Test_6_consonant_diacritic   
21         Test_7_grapheme_root   
22       Test_7_vowel_diacritic   
23   Test_7_consonant_diacritic   
24         Test_8_grapheme_root   
25       Test_8_vowel_diacritic   
26   Test_8_consonant_diacritic   
27         Test_9_gr